Задание №4. Вероятностная оценка запасов методом Монте-Карло

In [8]:
import numpy as np
import random
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

Функции для расчета STOIIP и создания списка значений с нормальным распределением

In [9]:
def calculate_stoiip(area,
                     effective_thickness,
                     porosity,
                     oil_formation_factor,
                     water_saturation=None,
                     oil_saturation=None):
    """
    :param area:                    squared metres * 10^6
    :param effective_thickness:     metres
    :param porosity:                fraction
    :param oil_formation_factor:    dimensionless
    :param water_saturation:        fraction
    :param oil_saturation:          fraction
    :return:
    """
    oil_saturation = 1 - water_saturation if oil_saturation is None else oil_saturation
    return area * 10 ** 6 * effective_thickness * porosity * oil_saturation / oil_formation_factor


def create_standard_distribution(start_value, end_value, number_of_values) -> list:
    sequence_list = np.linspace(start_value, end_value, number_of_values)
    mean_value = np.mean(sequence_list)
    standard_deviation = (mean_value - start_value) / 4
    prob_density = np.random.normal(mean_value, standard_deviation, number_of_values).tolist()
    return prob_density


Функция, получающая рандомные значения из списков

In [10]:
def get_random_values(area: list,
                      effective_thickness: list,
                      porosity: list,
                      oil_saturation: list,
                      oil_formation_factor: list) -> dict:
    return {'area': random.choice(area),
            'effective_thickness': random.choice(effective_thickness),
            'porosity': random.choice(porosity),
            'oil_saturation': random.choice(oil_saturation),
            'oil_formation_factor': random.choice(oil_formation_factor)}

Генерация данных для каждой переменной STOIIP

In [11]:
number_values = 10_000  # Number of values for each list
# Получение стандартного распределения для площади залежи
area_list = create_standard_distribution(start_value=100.0,
                                         end_value=1600.0,
                                         number_of_values=number_values)
# Получение стандартного распределения для эффективной мощности пласта
effective_thickness_list = create_standard_distribution(start_value=0.5,
                                                        end_value=10.0,
                                                        number_of_values=number_values)
# Получение стандартного распределения для пористости
porosity_list = create_standard_distribution(start_value=0.01,
                                             end_value=0.35,
                                             number_of_values=number_values)
# Получение стандартного распределения для коэффициента насыщения нефтью
oil_saturation_list = create_standard_distribution(start_value=0.20,
                                                   end_value=0.90,
                                                   number_of_values=number_values)
# Получение стандартного распределения для объемного коэффициента нефти
oil_formation_factor_list = create_standard_distribution(start_value=1.01,
                                                         end_value=1.4,
                                                         number_of_values=number_values)
# Сохранение данных в словарь
histogram_dict = {'Area distribution': area_list,
                  'Effective thickness distribution': effective_thickness_list,
                  'Porosity distribution': porosity_list,
                  'Oil saturation distribution': oil_saturation_list,
                  'Oil formation factor distribution': oil_formation_factor_list,
                  }


Визуализация распределений для каждой переменной STOIIP

In [12]:
figures = make_subplots(rows=5, cols=1, subplot_titles=list(histogram_dict.keys()))
for num, histogram in enumerate(histogram_dict):
    figures.append_trace(go.Histogram(x=histogram_dict.get(histogram), name=histogram),
                         row=num + 1, col=1)
figures.update_layout(height=600, width=800, title_text="STOIIP values standard distribution", title_x=0.3)
figures.show()

Вероятностная оценка запасов методом Монте-Карло

In [13]:
iterations_number = 10_000
stoiip_list = []
for iteration in range(iterations_number):
    generated_values = get_random_values(area=area_list,
                                         effective_thickness=effective_thickness_list,
                                         porosity=porosity_list,
                                         oil_saturation=oil_saturation_list,
                                         oil_formation_factor=oil_formation_factor_list)
    stoiip_list.append(calculate_stoiip(area=generated_values.get('area'),
                                        effective_thickness=generated_values.get('effective_thickness'),
                                        porosity=generated_values.get('porosity'),
                                        oil_saturation=generated_values.get('oil_saturation'),
                                        oil_formation_factor=generated_values.get('oil_formation_factor')
                                        )
                       )
fig = px.ecdf(pd.DataFrame(stoiip_list, columns=['STOIIP']), x="STOIIP")
fig.update_layout(title_text="Вероятностная оценка запасов методом Монте-Карло",
                  title_x=0.5)
fig.show()